# 5.5 Memory on LLMChain

### 학습목표: memory를 chain에 꽂는 방법 배우기. 또한 두 종류의 chain을 사용해서 꽂는 방법도 배우기

#### LLMChain

LLMChain은 다음 세 가지 주요 구성 요소를 결합하여 하나의 실행 흐름을 만듭니다:

##### 주요 구성 요소

1. **LLM (Language Model)**
   - 실제 대화를 처리하는 언어 모델
   - 예: GPT-3, GPT-4 등

2. **Memory**
   - 대화 기록을 저장하고 관리
   - 예: ConversationBufferMemory, ConversationSummaryMemory 등

3. **Prompt Template**
   - 언어 모델에게 전달할 프롬프트의 형식을 정의
   - 예: PromptTemplate.from_template("{question}")

##### 작동 방식
```python
   chain = LLMChain(
      llm=ChatOpenAI(), # 언어 모델
      memory=ConversationBufferMemory(), # 메모리
      prompt=PromptTemplate.from_template("{question}") # 프롬프트 템플릿
   )

   response = chain.predict(question="Hello!")
```
      이렇게 만들어진 체인은 입력 → 메모리 참조 → LLM 처리 → 출력의 흐름을 자동으로 관리합니다.

      즉 LLMChin은 Langchain에서 이미 만들어놓은 사용하기 쉬운 chain인 것이다.


In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True # 이 값을 True로 하면 chain의 프롬프트 로그들을 확인할 수 있다.
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:
My name is Nico

> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [9]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


", the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic and exciting place to call home."

In [10]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

코드를 보면 프롬프트 히스토리가 우리가 AI에게 전달하는 프롬프트에 계속 추가되진 않았다. 이것을 추가하는게 우리가 할 일이다.

근데 메모리는 계속 업데이트 되고 있었다. 다음 코드를 보자.

In [11]:
memory.load_memory_variables({})

{'history': 'System: The human introduces themselves as Nico. The AI responds by saying, "Nice to meet you, Nico! How can I assist you today?"\nHuman: I live in Seoul\nAI: , the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic palaces. I love exploring the city\'s neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic and exciting place to call home.\nHuman: What is my name?\nAI: I\'m sorry, I do not have access to personal information such as your name.'}

이렇게 요약한 내용을 메모리에 저장하고있는 것은 잘 작동하고 있었다.

메모리를 AI에게 전달해주기 위한 작업을 해보자.

In [15]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history" # 여기서 지정한 키 이름을 template의 변수로 사용. 
    # 즉, memory_key는 메모리의 내용을 프롬프트 템플릿에서 어떤 변수명으로 참조할지 지정하는 파라미터이다.
    # 이렇게 하면 메모리에 저장된 대화 내용이 chat_history라는 키로 프롬프트에 삽입된다.
    # 만약 memory_key="conversation" 으로 했다면, 프롬프트 에서도 {conversation}으로 참조해야 한다.
    # 즉, memory_key는 메모리와 프롬프트 템플릿을 연결하는 다리 역할을 한다.
)


template = """
    You are a helpful AI talking to a human.

    {chat_history} 
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True # 이 값을 True로 하면 chain의 프롬프트 로그들을 확인할 수 있다.
)

chain.predict(question="My name is Nico")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

     
    Human:My name is Nico
    You:


> Finished chain.


'Hello Nico! How can I assist you today?'

In [16]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today? 
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with information or tasks related to Seoul?"

위의 출력을 보면 알 수 있겠지만, 프롬프트에 우리의 대화 기록이 남겨져 있다.

In [17]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul? 
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'

우리의 대화 내용을 잘 기억해서 출력을 하는 모습이다.

또한 입력을 더 넣어서 대화 내용을 더 넣어서 메모리에 저장해보면 알겠지만, 우리의 대화내용이 길어지면, 오래된 내용은 자동으로 요약해서 저장한다는 것도 알 수 있다.